In [0]:
!unzip -q yelp_ratings.csv.zip

In [0]:
import spacy

In [0]:
from spacy.util import minibatch
import random

In [3]:
spacy.require_gpu()

True

In [0]:
import pandas as pd
import numpy as np

In [0]:
df = pd.read_csv('yelp_ratings.csv')

In [7]:
df.head()

,text,stars,sentiment
0,Total bill for this horrible service? Over $8G...,1.0,0
1,I *adore* Travis at the Hard Rock's new Kelly ...,5.0,1
2,I have to say that this office really has it t...,5.0,1
3,Went in for a lunch. Steak sandwich was delici...,5.0,1
4,Today was my second out of three sessions I ha...,1.0,0


In [0]:
texts = df.text.values

In [0]:
labels = [{'POSITIVE': bool(y),'NEGATIVE': not bool(y)} for y in df.sentiment.values]

In [0]:
idx = int(len(df)*0.85)

In [0]:
trainx = texts[:idx]
trainy = [{'cats':label} for label in labels[:idx]]
testx = texts[idx:]
testy = [{'cats':label} for label in labels[idx:]]

In [0]:
nlp = spacy.blank('en')

In [0]:
model = nlp.create_pipe('textcat',config = {'architecture':'bow','exclusive_classes':True})

In [0]:
nlp.add_pipe(model)

In [15]:
model.add_label('NEGATIVE')
model.add_label('POSITIVE')

1

In [0]:
def pred(model,texts):
  doc = [nlp.tokenizer(text) for text in texts]
  model = model.get_pipe('textcat')
  scores,_ = model.predict(doc)
  preds = scores.argmax(1)
  return preds

In [0]:
p = pred(nlp,testx)

In [50]:
eval(nlp,testx,testy)

0.948502994011976

In [0]:
def eval(model,texts,labels):
  preds = pred(model,texts)
  lb = [int(l['cats']['POSITIVE']) for l in labels]
  #print(lb)
  #preds = preds.tolist()
  #lb = np.array(lb)
  #preds = np.array(preds)
  preds = preds.tolist()
  lb = np.array(lb)
  preds = np.array(preds)
  acc = lb==preds
  acc = acc.mean()
  return acc

In [51]:
opt = nlp.begin_training()
train = list(zip(trainx,trainy))
losses = {}
for e in range(10):
  random.shuffle(train)
  batches = minibatch(train)
  for batch in batches:
    x,y = zip(*batch)
    nlp.update(x,y,sgd = opt,losses = losses)
  #lbs = [int(l['cats']['POSITIVE']) for l in testy]
  acc = eval(nlp,testx,testy)
  print(acc)

0.949251497005988
0.9479041916167664
0.947754491017964


KeyboardInterrupt: ignored